Refer to https://github.com/xiaoweiChen/Cpp_Concurrency_In_Action/blob/master/content/chapter5/5.3-chinese.md

## 5.3 同步操作和强制排序

假设你有两个线程，
- 一个向数据结构中填充数据，
- 另一个读取数据结构中的数据。

为了避免恶性条件竞争，
- 第一个线程设置一个标志，用来表明数据已经准备就绪，
- 并且第二个线程在这个标志设置前不能读取数据。

下面的程序清单就是这样的情况。

#### 清单5.2 不同线程对数据的读写

In [1]:
#include <vector>
#include <atomic>
#include <iostream>
#include <thread>

In [2]:
{
    std::vector<int> data;
    std::atomic<bool> data_ready(false);

    auto reader_thread = [&data, &data_ready] {
        while (!data_ready.load())  // 1
        {
            std::this_thread::sleep_for(std::chrono::milliseconds(1));
        }
        std::cout << "The answer=" << data[0] << "\n";  // 2
    };

    auto writer_thread = [&data, &data_ready] {
        data.push_back(42);  // 3
        data_ready = true;  // 4
    };
    
    auto t1 = std::thread(reader_thread);
    auto t2 = std::thread(writer_thread);
    t1.join();
    t2.join();
}

The answer=42


先把等待数据的低效循环①放在一边（你需要这个循环，否则想要在线程间共享数据就是不切实际的：数据的每一项都必须是原子的）。你已经知道，当非原子读②和写③对同一数据结构进行无序访问时，将会导致未定义行为的发生，因此**这个循环就是确保访问循序被严格的遵守的**。

强制访问顺序是由对`std::atomic<bool>`类型的data_ready变量进行操作完成的

### 5.3.1 同步发生

**“同步发生”只能在原子类型之间进行操作。**例如对一个数据结构进行操作(对互斥量上锁)，如果数据结构包含有原子类型，并且操作内部执行了一定的原子操作，那么这些操作就是同步发生关系。从根本上说，这种关系只能来源于对原子类型的操作。

“同步发生”的基本想法是：
- 在变量x进行适当标记的原子写操作W，同步与对x进行适当标记的原子读操作，读取的是W操作写入的内容；
- 或是在W之后，同一线程上的原子写操作对x写入的值；
- 亦或是任意线程对x的一系列原子读-改-写操作(例如，fetch_add()或compare_exchange_weak())。这里，第一个线程读取到的值是W操作写入的(详见5.3.4节)。

先将“适当的标记”放在一边，因为所有对原子类型的操作，默认都是适当标记的。这实际上就是：如果线程A存储了一个值，并且线程B读取了这个值，线程A的存储操作与线程B的载入操作就是同步发生的关系，如同清单5.2所示的那样。

我确信你假设过，所有细微的差别都在“适当的标记”中。C++内存模型允许为原子类型提供各种约束顺序，并且这个标记我们已经提过了。内存排序的各种选项和它们如何与同步发生的关系，将会在5.3.3节中讨论。

### 5.3.2 先行发生

“先行发生”关系
- 是一个程序中，基本构建块的操作顺序；它指定了某个操作去影响另一个操作。
- 对于单线程来说，就简单了：当一个操作排在另一个之后，那么这个操作就是先行执行的。这意味着，如果源码中操作A发生在操作B之前，那么A就先行于B发生。
- 你可以回看清单5.2：对data的写入③先于对data_ready④的写入。
- 如果操作在同时发生，因为操作间无序执行，通常情况下，它们就没有先行关系了。这就是另一种排序未被指定的情况。下面的程序会输出“1，2”或“2，1”，因为两个get_num()的执行顺序未被指定。

In [3]:
#include <iostream>

namespace n1 {
    void foo(int a, int b)
    {
        std::cout << a << ", " << b << std::endl;
    }

    int get_num()
    {
        static int i = 0;
        return ++i;
    }
}

In [4]:
{
    using namespace n1;
    foo(get_num(), get_num());  // 无序调用get_num()
}

1, 2


某系情况下，操作在**单一声明中是可测序的**，
- 例如，逗号操作符的使用，
- 或一个表达式的结果作为一个参数传给另一个表达式。

**但在通常情况下，操作在单一声明中是不可测序的**，所以对其无法先行安排顺序(也就没有先行发生了)。当然，**所有操作在一个声明中先行于在下一个声明中的操作**。

这是单个线程中的情况，多个线程中呢？如果一个线程中的操作A先于另一个线程中的操作B发生（inter-thread happens-before），那么A先于（happens-before）B发生。这并没有什么帮助：徒劳引入一个新的概念：inter-thread happens-before。但这对于编写多线程代码是尤为重要的。

从基本层面上讲，线程间的先行比较简单，并且依赖与同步关系(详见5.3.1节):
- 如果操作A在一个线程上，与另一个线程上的操作B同步，那么A就线程间先行于B。
- 这同样是一个传递关系：如果A线程间先行于B，并且B线程间先行于C，那么A就线程间先行于C。你可以回看一下清单5.2。

线程间先行可以与排序先行关系相结合：
- 如果操作A排序先行于操作B，并且操作B线程间先行于操作C，那么A线程间先行于C。
- 同样的，如果A同步于B，并且B排序先于C，那么A线程间先行于C。
- 两者的结合，意味着当你对数据进行一系列修改(单线程)时，为线程后续执行C，只需要对可见数据进行一次同步。

这些是线程间强制排序操作的关键规则，也是让清单5.2正常运行的因素。并在数据依赖上有一些细微的差别，你马上就会看到。为了让你理解这些差别，需要讲述一下原子操作使用的内存排序标签，以及这些标签和同步发生之间的联系。

# 5.3.3 原子操作的内存顺序

这里有六个内存序列选项可应用于对原子类型的操作：
- memory_order_relaxed, 
- memory_order_consume, 
- memory_order_acquire, 
- memory_order_release, 
- memory_order_acq_rel, 
- 以及memory_order_seq_cst。

除非你为特定的操作指定一个序列选项，要不内存序列选项对于**所有原子类型默认都是memory_order_seq_cst**。

虽然有六个选项，**但是它们仅代表三种内存模型：**
- 排序一致序列(sequentially consistent)，
- 获取-释放序列(memory_order_consume, memory_order_acquire, memory_order_release和memory_order_acq_rel)，
- 和自由序列(memory_order_relaxed)。

这些不同的内存序列模型，在**不同的CPU架构下，功耗是不一样的**。
- 例如，基于处理器架构的可视化精细操作的系统，比起其他系统，添加的同步指令可被排序一致序列使用(在获取-释放序列和自由序列之前)，或被获取-释放序列调用(在自由序列之前)。**如果这些系统有多个处理器，这些额外添加的同步指令可能会消耗大量的时间**，从而降低系统整体的性能。
- 另一方面，CPU使用的是**x86或x86-64架构**(例如，使用Intel或AMD处理器的台式电脑)，使用这种架构的CPU不需要任何对获取-释放序列添加额外的指令(没有保证原子性的必要了)，并且，即使是排序一致序列，对于加载操作也不需要任何特殊的处理，**不过在进行存储时，有点额外的消耗。**

不同种类的内存序列模型，**允许专家**利用其提升与更细粒度排序相关操作的性能。当**默认使用排序一致序列(相较于其他序列，它是最简单的)时，对于在那些不大重要的情况下是有利的**。

选择使用哪个模型，或为了了解与序列相关的代码，为什么选择不同的内存模型，是需要了解一个重要的前提，那就是**不同模型是如何影响程序的行为**。让我们来看一下选择每个操作序列和同步相关的结果。

#### 排序一致队列(memory_order_seq_cst)

默认序列命名为排序一致，是因为程序中的行为从任意角度去看，序列顺序都保持一致。
- 如果原子类型实例上的所有操作都是序列一致的，那么一个多线程程序的行为，就以某种特殊的排序执行，好像单线程那样。
- 这是目前来看，最容易理解的内存序列，这也就是将其设置为默认的原因：所有线程都必须了解，不同的操作也遵守相同的顺序。因为其简单的行为，可以使用原子变量进行编写。
- 通过不同的线程，你可以写出所有序列上可能的操作，这样就可以消除那些不一致，以及验证你代码的行为是否与预期相符。
- 这也就意味着，所有操作都不能重排序；如果你的代码，在一个线程中，将一个操作放在另一个操作前面，那么这个顺序就必须让其他所有的线程所了解。

从同步的角度看，对于同一变量，排序一致的存储操作同步相关于同步一致的载入操作。这就提供了一种对两个(以上)线程操作的排序约束，但是排序一致的功能要比排序约束大的多。所以，对于使用排序一致原子操作的系统上的任一排序一致的原子操作，都会在对值进行存储以后，再进行加载。清单5.4就是这种一致性约束的演示。这种约束不是线程在自由内存序列中使用原子操作；这些线程依旧可以知道，操作以不同顺序排列，所以你必须使用排序一致操作，去保证在多线的情况下有加速的效果。